<a href="https://colab.research.google.com/github/Gayaz2000/Unsloth-DPOTrainer/blob/main/Unsloth_DPO_LoRA_FT_Llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/

##DPO FineTuning

In [ ]:
#unsloth/Llama-3.2-1B-Instruct
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


##Loading Model

In [ ]:
from unsloth import FastLlamaModel
#from transformers import BitAndBytesConfig
import torch

max_seq_length = 4096
dtype = None
load_in_4bit= True

#bnb_config = BitAndBytesConfig(
#    load_in_4bit= True,
#    bnb_4bit_use_double_quant= True,
#    bnb_4bit_quant_type= "nf4",
#    bnb_4bit_compute_dtype= torch.bfloat16
#)

model, tokenizer = FastLlamaModel.from_pretrained(
    model_name= "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length= max_seq_length,
    load_in_4bit= load_in_4bit,
    #bnb_config= bnb_config,
    dtype= dtype,
)

==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


LoRA Config

In [ ]:
model = FastLlamaModel.get_peft_model(
    model= model,
    r= 8,
    lora_alpha= 16,
    target_modules= ["q_proj", "v_proj","v_proj","up_proj", "down_proj", "o_proj"],
    lora_dropout= 0,
    bias= "none",
    #task_type= "CAUSAL_LM",
    use_gradient_checkpointing = "unsloth",
    random_state = 3047,
    loftq_config = None,
    use_rslora = False,
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.6.2 patched 16 layers with 0 QKV layers, 16 O layers and 0 MLP layers.


In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linea

##Data Prep

In [ ]:
import os
import re
from typing import List, Literal, Optional

from datasets import DatasetDict, concatenate_datasets, load_dataset, load_from_disk
from datasets.builder import DatasetGenerationError

DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"


def apply_chat_template(
    example,
    tokenizer,
    task: Literal["sft", "generation", "rm", "dpo"] = "sft",
    assistant_prefix="<|assistant|>\n",
):
    def _strip_prefix(s, pattern):
        # Use re.escape to escape any special characters in the pattern
        return re.sub(f"^{re.escape(pattern)}", "", s)

    if task == "dpo":
        if all(k in example.keys() for k in ("chosen", "rejected")):
            # Compared to reward modeling, we filter out the prompt, so the text is everything after the last assistant token
            prompt_messages = [
                [msg for msg in example["chosen"] if msg["role"] == "user"][0]
            ]
            # Insert system message
            if example["chosen"][0]["role"] != "system":
                prompt_messages.insert(0, {"role": "system", "content": ""})
            else:
                prompt_messages.insert(0, example["chosen"][0])
            # TODO: handle case where chosen/rejected also have system messages
            chosen_messages = example["chosen"][1:]
            rejected_messages = example["rejected"][1:]
            example["text_chosen"] = tokenizer.apply_chat_template(
                chosen_messages, tokenize=False
            )
            example["text_rejected"] = tokenizer.apply_chat_template(
                rejected_messages, tokenize=False
            )
            example["text_prompt"] = tokenizer.apply_chat_template(
                prompt_messages, tokenize=False, add_generation_prompt=True
            )
            example["text_chosen"] = _strip_prefix(
                example["text_chosen"], assistant_prefix
            )
            example["text_rejected"] = _strip_prefix(
                example["text_rejected"], assistant_prefix
            )
        else:
            raise ValueError(
                f"Could not format example as dialogue for `dpo` task! Require `[chosen, rejected]` keys but found {list(example.keys())}"
            )
    else:
        raise ValueError(
            f"Task {task} not supported, please ensure that the provided task is one of {['sft', 'generation', 'rm', 'dpo']}"
        )
    return example


In [ ]:
def mix_datasets(dataset_mixer: dict, splits: Optional[List[str]] = None, shuffle=True) -> DatasetDict:
    """
    Loads and mixes datasets according to proportions specified in `dataset_mixer`.

    Args:
        dataset_mixer (`dict`):
            Dictionary containing the dataset names and their training proportions. By default, all test proportions are 1.
        splits (Optional[List[str]], *optional*, defaults to `None`):
            Dataset splits to load and mix. Assumes the splits exist in all datasets and have a `train_` or `test_` prefix.
        shuffle (`bool`, *optional*, defaults to `True`):
            Whether to shuffle the training and testing/validation data.
    """
    raw_datasets = DatasetDict()
    raw_train_datasets = []
    raw_val_datasets = []
    fracs = []
    for ds, frac in dataset_mixer.items():
        fracs.append(frac)
        for split in splits:
            try:
                # Try first if dataset on a Hub repo
                dataset = load_dataset(ds, split=split)
            except DatasetGenerationError:
                # If not, check local dataset
                dataset = load_from_disk(os.path.join(ds, split))

            if "train" in split:
                raw_train_datasets.append(dataset)
            elif "test" in split:
                raw_val_datasets.append(dataset)
            else:
                raise ValueError(
                    f"Split type {split} not recognized as one of test or train."
                )

    if any(frac < 0 for frac in fracs):
        raise ValueError("Dataset fractions cannot be negative.")

    if len(raw_train_datasets) > 0:
        train_subsets = []
        for dataset, frac in zip(raw_train_datasets, fracs):
            train_subset = dataset.select(range(int(frac * len(dataset))))
            train_subsets.append(train_subset)
        if shuffle:
            raw_datasets["train"] = concatenate_datasets(train_subsets).shuffle(seed=42)
        else:
            raw_datasets["train"] = concatenate_datasets(train_subsets)
    # No subsampling for test datasets to enable fair comparison across models
    if len(raw_val_datasets) > 0:
        if shuffle:
            raw_datasets["test"] = concatenate_datasets(raw_val_datasets).shuffle(
                seed=42
            )
        else:
            raw_datasets["test"] = concatenate_datasets(raw_val_datasets)

    if len(raw_datasets) == 0:
        raise ValueError(
            f"Dataset {dataset_mixer} not recognized with split {split}. Check the dataset has been correctly formatted."
        )

    return raw_datasets

In [ ]:
def get_datasets(data_config: dict,splits: List[str] = ["train", "test"],shuffle: bool = True,) -> DatasetDict:
    """
    Loads one or more datasets with varying training set proportions.

    Args:
        data_config (`DataArguments` or `dict`):
            Dataset configuration and split proportions.
        splits (`List[str]`, *optional*, defaults to `['train', 'test']`):
            Dataset splits to load and mix. Assumes the splits exist in all datasets and have a `train_` or `test_` prefix.
        shuffle (`bool`, *optional*, defaults to `True`):
            Whether to shuffle the training and testing/validation data.

    Returns
        [`DatasetDict`]: The dataset dictionary containing the loaded datasets.
    """

    if type(data_config) is dict:
        # Structure of the input is:
        #     dataset_mixer = {
        #             "dataset1": 0.5,
        #             "dataset1": 0.3,
        #             "dataset1": 0.2,
        #         }
        dataset_mixer = data_config
    else:
        raise ValueError(f"Data config {data_config} not recognized.")

    raw_datasets = mix_datasets(dataset_mixer, splits=splits, shuffle=shuffle)
    return raw_datasets

In [ ]:
# @title load and tranform dataset
raw_datasets = get_datasets(
    {"HuggingFaceH4/ultrafeedback_binarized" : 0.005}, # 0.5% sampled
    splits = ["train_prefs", "test_prefs"],
)

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 305
    })
    test: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 2000
    })
})

In [ ]:
raw_datasets["train"]

Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 305
})

In [ ]:
raw_datasets["train"][0]

{'prompt': 'Thoreau’s metaphors are highly visual. Though they’re clever and original, they aren’t far-fetched. Thoreau takes his comparisons from nature and from other things with which he and his audience are familiar. To be sure you understand Thoreau’s figures of speech, paraphrase the following metaphor. Do his metaphors apply to readers today?\n\n“If a man does not keep pace with his companions, perhaps it is because he hears a different drummer. Let him step to the music which he hears, however measured or far away.”',
 'prompt_id': '4fbd868228cba2b377fe495101887e8f0d0e58fdf9b487b08b7a637ca34151dd',
 'chosen': [{'content': 'Thoreau’s metaphors are highly visual. Though they’re clever and original, they aren’t far-fetched. Thoreau takes his comparisons from nature and from other things with which he and his audience are familiar. To be sure you understand Thoreau’s figures of speech, paraphrase the following metaphor. Do his metaphors apply to readers today?\n\n“If a man does not

In [ ]:
column_names = list(raw_datasets["train"].features)

In [ ]:
column_names

['prompt',
 'prompt_id',
 'chosen',
 'rejected',
 'messages',
 'score_chosen',
 'score_rejected']

In [ ]:
formatted_datasets = raw_datasets.map(
    apply_chat_template,
    fn_kwargs = {"tokenizer": tokenizer, "task": "dpo"},
    num_proc = 12,
    remove_columns = column_names,
    desc = "Formatting comparisons with prompt template",
)

Formatting comparisons with prompt template (num_proc=12):   0%|          | 0/305 [00:00<?, ? examples/s]

Formatting comparisons with prompt template (num_proc=12):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
formatted_datasets["train"]

Dataset({
    features: ['text_chosen', 'text_rejected', 'text_prompt'],
    num_rows: 305
})

In [ ]:
# Replace column names with what TRL needs, text_chosen -> chosen and text_rejected -> rejected
for split in ["train", "test"]:
    formatted_datasets[split] = formatted_datasets[split].rename_columns(
        {"text_prompt": "prompt", "text_chosen": "chosen", "text_rejected": "rejected"}
    )

In [ ]:
import pprint

row = formatted_datasets["train"][8]
pprint.pprint(row["prompt"])

('<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n'
 '\n'
 'Cutting Knowledge Date: December 2023\n'
 'Today Date: 14 Jun 2025\n'
 '\n'
 '<|eot_id|><|start_header_id|>user<|end_header_id|>\n'
 '\n'
 'Describe a possible solution to the environmental issue of air '
 'pollution.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n'
 '\n')


In [ ]:
pprint.pprint(row["chosen"])

('<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n'
 '\n'
 'Cutting Knowledge Date: December 2023\n'
 'Today Date: 14 Jun 2025\n'
 '\n'
 '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n'
 '\n'
 'One of the most effective solutions to the environmental issue of air '
 'pollution is promoting and investing in renewable energy sources. '
 'Traditional energy sources like coal and oil produce large amounts of '
 'greenhouse gases that contribute greatly to air pollution. Renewable energy '
 'sources like wind, solar, and hydropower sources do not produce greenhouse '
 'gases. They use clean energy sources that do not harm the environment and do '
 'not contribute to air pollution. \n'
 '\n'
 'Another solution is improving our public transportation systems. Encouraging '
 'individuals to use public transportation, cycling, walking, or carpooling '
 'instead of their personal vehicles can greatly reduce emissions. This is '
 'especially helpful in urban areas where tra

In [ ]:
pprint.pprint(row["rejected"])

('<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n'
 '\n'
 'Cutting Knowledge Date: December 2023\n'
 'Today Date: 14 Jun 2025\n'
 '\n'
 '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n'
 '\n'
 "Hello! As a helpful and respectful assistant, I'm here to provide you with "
 'information and answers that are safe, accurate, and positive. I understand '
 "the importance of addressing environmental issues, and I'm happy to help you "
 'explore possible solutions to air pollution.\n'
 '\n'
 'Air pollution is a pressing environmental issue that affects not only the '
 'environment but also human health. It can cause respiratory problems, '
 'cardiovascular diseases, and even cancer. To address this issue, there are '
 'several possible solutions that can be implemented at the individual, '
 'community, and government levels.\n'
 '\n'
 'One possible solution is to use cleaner energy sources such as wind, solar, '
 'and hydroelectric power. These sources of energy are ren

##Trainig DPO

In [ ]:
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

In [ ]:
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported

dpo_config = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 2,
        learning_rate = 5e-6,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        #remove_unused_columns=False
    )

dpo_trainer = DPOTrainer(
    model = model,
    beta = 0,
    tokenizer = tokenizer,
    args = dpo_config,
    train_dataset = formatted_datasets["train"],
    #eval_dataset = raw_datasets["test"],
    max_length = 1024,
    max_prompt_length = 512,
)

Extracting prompt in train dataset (num_proc=2):   0%|          | 0/305 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/305 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/305 [00:00<?, ? examples/s]

In [ ]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 305 | Num Epochs = 2 | Total steps = 78
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 3,997,696/1,000,000,000 (0.40% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
1,0.693100,0.000000,0.000000,0.000000,0.000000,-310.176056,-270.560394,1.129909,0.876811,0,0,0,0
2,0.693100,0.000000,0.000000,0.000000,0.000000,-387.872742,-459.302551,1.205412,1.485961,No Log,No Log,No Log,No Log
3,0.693100,0.000000,0.000000,0.000000,0.000000,-489.722046,-352.156738,1.340209,1.275441,No Log,No Log,No Log,No Log
4,0.692800,0.002550,0.001871,0.625000,0.000678,-341.725647,-340.344421,1.239938,1.335207,No Log,No Log,No Log,No Log
5,0.692500,-0.000647,-0.001999,0.500000,0.001352,-379.573822,-393.386353,1.520163,1.196404,No Log,No Log,No Log,No Log
6,0.697500,-0.000256,0.008464,0.250000,-0.008720,-593.586365,-429.201935,1.239164,1.166272,No Log,No Log,No Log,No Log
7,0.694000,-0.001361,0.000301,0.625000,-0.001662,-742.011597,-574.039917,1.599991,1.461543,No Log,No Log,No Log,No Log
8,0.693300,-0.003159,-0.002783,0.500000,-0.000376,-448.942017,-352.138763,1.625336,1.459368,No Log,No Log,No Log,No Log
9,0.693700,-0.000177,0.000950,0.500000,-0.001127,-300.815643,-399.525970,1.471614,1.420258,No Log,No Log,No Log,No Log
10,0.693500,-0.001648,-0.001103,0.750000,-0.000544,-703.225586,-568.384521,1.086706,1.197680,No Log,No Log,No Log,No Log


TrainOutput(global_step=78, training_loss=0.664065607656271, metrics={'train_runtime': 479.8728, 'train_samples_per_second': 1.271, 'train_steps_per_second': 0.163, 'total_flos': 0.0, 'train_loss': 0.664065607656271, 'epoch': 2.0})

In [68]:
model.save_pretrained("DPOTrained-lora_llama3.21b")
tokenizer.save_pretrained("DPOTrained-lora_llama3.21b")

('DPOTrained-lora_llama3.21b/tokenizer_config.json',
 'DPOTrained-lora_llama3.21b/special_tokens_map.json',
 'DPOTrained-lora_llama3.21b/chat_template.jinja',
 'DPOTrained-lora_llama3.21b/tokenizer.json')

In [77]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
# Push to Hugging Face Hub
from transformers import AutoModelForCausalLM, AutoTokenizer

AutoModelForCausalLM.from_pretrained("DPOTrained-lora_llama3.21b").push_to_hub("Gayaz2000/DPOTrained-lora_llama3.21b")
AutoTokenizer.from_pretrained("DPOTrained-lora_llama3.21b").push_to_hub("Gayaz2000/DPOTrained-lora_llama3.21b")

HfHubHTTPError: (Request ID: Root=1-684d57f0-42a3765e4d68aeb10f4040d5;6d214c00-8449-4157-bb2b-1688a51d2ba5)

403 Forbidden: You don't have the rights to create a model under the namespace "Gayaz2000".
Cannot access content at: https://huggingface.co/api/repos/create.
Make sure your token has the correct permissions.

##Inferencing

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


I can try to provide a descriptive passage about a tall tower in the capital of France.

Imagine standing at the base of the Château de Fontainebleau, a majestic and historic tower situated in the capital of France, Paris. As you ascend the tower's imposing structure, you're greeted by a sweeping view of the City of Light, the Paris Basin, and the Seine River.

The tower itself, known as the Château de Fontainebleau, is one of the largest and most impressive buildings in the City, with a history dating back to the 17th century. The 12-story tower stands proud
